In [1]:
# The code was removed by Watson Studio for sharing.

# Coursera Capstone Project

### This notebook will be used for my final project at the end of the IBM Data Science Professional Certificate

#### Import the required libraries

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-OpenCE

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    widgetsnbextension-3.5.1   |   py37h89c1867_4         1.8 MB  conda-f

In [3]:
import pandas as pd
path = 'https://github.com/NickyRichards123/Coursera_Capstone/blob/643bbde7aaf8277749ec12f70a39424fbcebc3ac/doogal_london_postcodes.csv?raw=true '
dfpostcodes = pd.read_csv(path)

Import list of postcodes
make list into dataframe for postcode and neighbourhood name

In [4]:
dfpostcodes.head()

,Unnamed: 0,Southern districts
0,SE1,"Bermondsey, Borough, Southwark, Waterloo"
1,SE2,Abbey Wood
2,SE3,"Blackheath, Westcombe Park"
3,SE4,"Brockley, Crofton Park, Honor Oak Park"
4,SE5,Camberwell


In [5]:
dfpostcodes=dfpostcodes.rename(columns={"Unnamed: 0" : "PostcodeArea","Southern districts":"Neighbourhoods"})
dfpostcodes.head()

,PostcodeArea,Neighbourhoods
0,SE1,"Bermondsey, Borough, Southwark, Waterloo"
1,SE2,Abbey Wood
2,SE3,"Blackheath, Westcombe Park"
3,SE4,"Brockley, Crofton Park, Honor Oak Park"
4,SE5,Camberwell


In [6]:
dfpostcodes.sort_values('PostcodeArea', 
                      inplace=True, 
                      ascending=True)

Import csv file of postcode lat/lon
clean the file, get rid of surplus columns and rows
take average of postcodes to find centre of each neighbourhood
stitch lat lon with postcodes dataframe

In [7]:
dfpostcodes.head()

,PostcodeArea,Neighbourhoods
0,SE1,"Bermondsey, Borough, Southwark, Waterloo"
9,SE10,Greenwich
10,SE11,Lambeth
11,SE12,"Grove Park, Lee"
12,SE13,"Hither Green, Lewisham"


In [8]:
dfpostcodes.reset_index(drop=True, inplace=True)
dfpostcodes.head()

,PostcodeArea,Neighbourhoods
0,SE1,"Bermondsey, Borough, Southwark, Waterloo"
1,SE10,Greenwich
2,SE11,Lambeth
3,SE12,"Grove Park, Lee"
4,SE13,"Hither Green, Lewisham"


In [9]:
path2= 'https://github.com/NickyRichards123/Coursera_Capstone/blob/155a3b8f8c73268ed338e2e1f81d551ba0b8e3f2/london_postcodes-ons-postcodes-directory.csv?raw=true'
dflatlon = pd.read_csv(path2)

In [10]:
dflatlon.head()

,pcd,reg,lat,long
0,SE1 0AA,SE1,51.498708,-0.101772
1,SE1 0AB,SE1,51.504038,-0.095542
2,SE1 0AD,SE1,51.499568,-0.101016
3,SE1 0AE,SE1,51.498575,-0.104112
4,SE1 0AF,SE1,51.498206,-0.104098


### Discard the column with the detailed postcode

In [11]:
dflatlon=dflatlon.drop('pcd', axis=1)


### Rename columns to be consistent

In [12]:
dflatlon=dflatlon.rename(columns={"reg" : "PostcodeArea","lat":"latitude", "long":"longitude"})
dflatlon.head()

,PostcodeArea,latitude,longitude
0,SE1,51.498708,-0.101772
1,SE1,51.504038,-0.095542
2,SE1,51.499568,-0.101016
3,SE1,51.498575,-0.104112
4,SE1,51.498206,-0.104098


In [13]:
dflatlon.dtypes

PostcodeArea     object
latitude        float64
longitude       float64
dtype: object

### Combine each group of postcodes in each area, taking an average of the latitude and longitude values

In [14]:
dfgroup=dflatlon.groupby("PostcodeArea",as_index=False).mean()

In [15]:
dfgroup.head()

,PostcodeArea,latitude,longitude
0,SE1,51.498191,-0.090293
1,SE10,51.484002,0.000746
2,SE11,51.489510,-0.109062
3,SE12,51.446044,0.025921
4,SE13,51.459298,-0.010488


In [16]:
dfgroup.dtypes

PostcodeArea     object
latitude        float64
longitude       float64
dtype: object

### Join together the dataframe containing the list of postcodes with the dataframe containing latitude and longitude data

In [17]:
Londondata=pd.merge(dfgroup, dfpostcodes, how='inner', left_on='PostcodeArea', right_on='PostcodeArea',left_index=True)

#### Check that the dataframe has joined correctly

In [18]:
Londondata

,PostcodeArea,latitude,longitude,Neighbourhoods
0,SE1,51.498191,-0.090293,"Bermondsey, Borough, Southwark, Waterloo"
1,SE10,51.484002,0.000746,Greenwich
2,SE11,51.489510,-0.109062,Lambeth
3,SE12,51.446044,0.025921,"Grove Park, Lee"
4,SE13,51.459298,-0.010488,"Hither Green, Lewisham"
5,SE14,51.475470,-0.043112,"New Cross, New Cross Gate"
6,SE15,51.472734,-0.067061,"Nunhead, Peckham"
7,SE16,51.496560,-0.054826,"Rotherhithe, South Bermondsey, Surrey Docks"
8,SE17,51.488515,-0.094691,"Elephant & Castle, Walworth"
9,SE18,51.485410,0.071126,"Plumstead, Woolwich"


### Create a map of South London using average latitude and longitude values from the data set as the centre, with markers on the map showing the centre of each Postcode Area

In [19]:
latitude=Londondata['latitude'].mean()
longitude=Londondata['longitude'].mean()

In [20]:
# create map of South London using average latitude and longitude values from the data set
map_SLondon = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, pcd, neighborhood in zip(Londondata['latitude'], Londondata['longitude'], Londondata['PostcodeArea'], Londondata['Neighbourhoods']):
    label = '{}, {}'.format(neighborhood, pcd)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SLondon)  
    
map_SLondon

### Define Foursquare Credentials

In [21]:
CLIENT_ID = 'NFJ23WYXHZIJYO0AWKR3MVAO1NCCAMFSFQ2GZE3UWY1W0I3D' # your Foursquare ID
CLIENT_SECRET = 'SRZMYEIGS2YNJKL5O1PKFX44E2ZQIISRR2ZEORU3ON4AWZ1J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NFJ23WYXHZIJYO0AWKR3MVAO1NCCAMFSFQ2GZE3UWY1W0I3D
CLIENT_SECRET:SRZMYEIGS2YNJKL5O1PKFX44E2ZQIISRR2ZEORU3ON4AWZ1J


### Define a request to explore nearby venues and run it on the South London neighbourhoods

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhoods', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
SLondonvenues=getNearbyVenues(names=Londondata['Neighbourhoods'], 
                                 latitudes=Londondata['latitude'], 
                                 longitudes=Londondata['longitude'])

Bermondsey, Borough, Southwark, Waterloo
Greenwich
Lambeth
Grove Park, Lee
Hither Green, Lewisham
New Cross, New Cross Gate
Nunhead, Peckham
Rotherhithe, South Bermondsey, Surrey Docks
Elephant & Castle, Walworth
Plumstead, Woolwich


In [ ]:
print(SLondonvenues.shape)
SLondonvenues.head()

### Using One Hot encoding, normalise the data and group by neighbourhood to allow comparison between neighbourhoods

In [ ]:
# one hot encoding
SLon_onehot = pd.get_dummies(SLondonvenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SLon_onehot['Neighbourhoods'] = SLondonvenues['Neighbourhoods'] 

# move neighborhood column to the first column
fixed_columns = [SLon_onehot.columns[-1]] + list(SLon_onehot.columns[:-1])
SLon_onehot = SLon_onehot[fixed_columns]

SLon_onehot.head()

In [ ]:
print(SLon_onehot.shape)

In [ ]:
SLon_grouped = SLon_onehot.groupby('Neighbourhoods').mean().reset_index()
SLon_grouped.head()

### Define a new function to return the 15 most common venues in each postcode area

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhoods'] = SLon_grouped['Neighbourhoods']

for ind in np.arange(SLon_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SLon_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

## Run K-Means clustering to cluster the neighbourhoods into 10 clusters

In [ ]:
# set number of clusters
kclusters = 10

SLon_grouped_clustering = SLon_grouped.drop('Neighbourhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SLon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

Create another dataframe which shows the cluster details as well as the most common venues


In [ ]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
SLon_merged = Londondata

SLon_merged = SLon_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhoods'), on='Neighbourhoods')

SLon_merged.head()

### Create a map to visually display the clustering of South London Neighbourhoods

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SLon_merged['latitude'], SLon_merged['longitude'], SLon_merged['Neighbourhoods'], SLon_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Define the latitude and longitude data for Bedminster

In [ ]:
address = 'Bedminster, UK'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bedminster are {}, {}.'.format(latitude, longitude))

### Explore venues around Bedminster

In [ ]:
radius=1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    radius,
    LIMIT)

results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
nearby_venues.shape

### Run One Hot encoding to normalise the data and get numerical values to detail the venue types

In [ ]:
# one hot encoding
Bedm_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bedm_onehot['Neighbourhoods'] = 'Bedminster'

# move neighborhood column to the first column
fixed_columns = [Bedm_onehot.columns[-1]] + list(Bedm_onehot.columns[:-1])
Bedm_onehot = Bedm_onehot[fixed_columns]

Bedm_onehot.head()

In [ ]:
Bedm_grouped = Bedm_onehot.groupby('Neighbourhoods').mean().reset_index()
Bedm_grouped.head()


### Create a data frame detailing the 15 most common venues in Bedminster

In [ ]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
bedminster_venues_sorted = pd.DataFrame(columns=columns)
bedminster_venues_sorted['Neighbourhoods'] = Bedm_grouped['Neighbourhoods']

for ind in np.arange(Bedm_grouped.shape[0]):
    bedminster_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bedm_grouped.iloc[ind, :], num_top_venues)

bedminster_venues_sorted

### Add the Bedminster most common venues to the London most common venues dataframe

In [ ]:
SLonBedmVen = neighbourhoods_venues_sorted.append(bedminster_venues_sorted)
SLonBedmVen.tail()

In [ ]:
SLonBedmVen.drop('Cluster Labels', axis=1, inplace=True)
SLonBedmVen.head()

### Add the normalised Bedminster venues data in to the normalised South London venues data

In [ ]:

SLon_grouped.head()

In [ ]:
Bedm_grouped.head()

In [ ]:
allgroup=pd.concat([SLon_grouped,Bedm_grouped], ignore_index=True)
allgroup

In [ ]:
allgroup=allgroup.fillna(0)
allgroup

## Run K-Means clustering on the combined data, including Bedminster and South London 

In [ ]:
# set number of clusters
kclusters = 10

allgroup_clustering = allgroup.drop('Neighbourhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(allgroup_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

In [ ]:
SLonBedmVen.insert(0, 'Cluster Labels', kmeans.labels_)
SLonBedmVen.head()

In [ ]:
SLonBedmVen.tail()

## Create a map to display the cluster containing Bedminster

In [ ]:
address = 'Bedminster, UK'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


bedmdata= pd.DataFrame({"PostcodeArea":"BS3", "latitude":latitude, "longitude":longitude, "Neighbourhoods":"Bedminster"}, index=[0])

In [ ]:
bedmdata

In [ ]:
Alldata=Londondata.append(bedmdata, ignore_index=True)

In [ ]:
Alldata.tail()

In [ ]:
All_merged = Alldata

All_merged = All_merged.join(SLonBedmVen.set_index('Neighbourhoods'), on='Neighbourhoods')

All_merged.tail()

In [ ]:
BedmClust = All_merged.loc[All_merged['Cluster Labels']==0]
BedmClust

In [ ]:
latitude=Londondata['latitude'].mean()
longitude=Londondata['longitude'].mean()
# create map
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(BedmClust['latitude'], BedmClust['longitude'], BedmClust['Neighbourhoods'], BedmClust['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2